In [4]:
import numpy as np
import scipy.signal as signal


In [5]:
DATA_DIR = r"../data/processed/BP_npy/0625_264_corr_clean/p00/"


In [6]:
sig = np.load(DATA_DIR+"train.npy")
print(sig.shape)

(132268, 264, 2)


In [14]:
def downsampling(x):
    fs = 125  # サンプリング周波数
    N = 264   # サンプル数
    fc = 15.625  # カットオフ周波数

    # 時間軸の生成
    t = np.arange(N) / fs

    # サンプル信号の生成（例: サイン波）
    freq = 10  # 信号の周波数
    # x = np.sin(2 * np.pi * freq * t)

    # ローパスフィルターの設計
    nyquist = 0.5 * fs
    normal_cutoff = fc / nyquist
    b, a = signal.butter(4, normal_cutoff, btype='low', analog=False)

    # フィルタリング
    filtered_signal = signal.filtfilt(b, a, x)

    # ダウンサンプリング
    downsampled_signal = filtered_signal[:,::4]
    return downsampled_signal

In [9]:
def calc_diff(sig,px=256):
    diff1 = np.diff(sig,axis=1)
    diff2 = np.diff(diff1,axis=1)
    sig = sig[:,:px]
    diff1 = diff1[:,:px]
    ret = np.stack([sig,diff1,diff2],axis=-1)
    print(ret.shape)
    return ret

In [12]:
def run(file,px=256):
    sig = np.load(DATA_DIR+file+".npy")
    ppg = downsampling(sig[:,:,1])
    print(ppg.shape)
    ppg = calc_diff(ppg,px=px//4)
    abp = sig[:,:px,0]
    print(file,ppg.shape,abp.shape)
    np.save(DATA_DIR+file+"_ppg.npy",ppg)
    np.save(DATA_DIR+file+"_abp.npy",abp)

In [15]:
run("train")


(132268, 66)
(132268, 64, 3)
train (132268, 64, 3) (132268, 256)


In [16]:
run("validate")
run("test")

(7468, 66)
(7468, 64, 3)
validate (7468, 64, 3) (7468, 256)
(16218, 66)
(16218, 64, 3)
test (16218, 64, 3) (16218, 256)
